<a href="https://colab.research.google.com/github/chminPark/ml-python/blob/master/%EC%8B%A4%EC%8A%B5_%EB%B0%98%EB%8F%84%EC%B2%B4%EC%9D%B4%EB%AF%B8%EC%A7%80%EB%B6%84%EB%A5%98_CNN_%5B%EC%A1%B0%EA%B5%90%EC%9A%A9%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1nOUWByW6ifnd_P-Jgd4mbNkAT-jGzpWC
!mkdir 'semicon'
!unzip -q semiconductor_dataset.zip -d semicon
!rm semiconductor_dataset.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1nOUWByW6ifnd_P-Jgd4mbNkAT-jGzpWC
To: /content/semiconductor_dataset.zip
100% 10.3M/10.3M [00:00<00:00, 214MB/s]
mkdir: cannot create directory ‘semicon’: File exists
replace semicon/abnorm_1/1-4.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
import csv
import os
import pandas as pd
"""
Args:
  directory_string: 이미지가 저장되어 있는 폴더 Path
  output_csv_name: csv 파일 이름
Returns:
  csv file
"""
def build_csv(directory_string, output_csv_name):

    directory = directory_string
    class_list = os.listdir(directory)
    class_list.sort()


    with open(output_csv_name, 'w', newline='') as csvfile:

        ### 실습 : CSV 파일 Object 생성
        writer = csv.writer(csvfile, delimiter=',')
        ############################################
        writer.writerow(['file_name', 'file_path', 'class_name', 'class_index']) # CSV의 column 이름을 지정

        ###### 각 folder에 들어가서 각 이미지의 이름을 가져옴
        for class_name in class_list:
          class_path = os.path.join(directory, class_name)
          file_list = os.listdir(class_path) # 해당 파일 내부의 이미지를 확보
          for file_name in file_list:
              file_path = os.path.join(directory, class_name, file_name) #concatenate class folder dir, class name and file name
              writer.writerow([file_name, file_path, class_name, class_name.split("_")[1]]) #write the file path and class name to the csv file
        #############################

    return

train_folder = os.path.join(os.getcwd(), 'semicon')
build_csv(train_folder, 'train.csv')
train_df = pd.read_csv('train.csv')


# Custom데이터를 활용하기 위한 Dataset을 선언
- pytorch의 `dataloader`를 이용하기 위해서는 `torch.utils.data.Dataset` 클래스를 상속한 클래스의 선언이 필요하다.
- 클래스 내에 `__init__`, `__getitem__`, `__len__`의 3개의 메소드를 선언하여 오버라이드한다.

In [ ]:
import cv2
import torch
from torch.utils.data import Dataset

class semiconductorDataset(Dataset): # inheritin from Dataset class

    def __init__(self, csv_file, root_dir="", transform=None):
        self.annotation_df = pd.read_csv(csv_file)
        self.root_dir = root_dir # root directory of images, leave "" if using the image path column in the __getitem__ method
        self.transform = transform

    def __len__(self):
        return len(self.annotation_df) # return length (numer of rows) of the dataframe

    ############ 실습 : annotation_df에 있는 이미지를 읽어 들어서, 변경한후
    ############        읽어 들인 값을 return 하는 함수를 작성한다.
    def __getitem__(self, idx):

        ##### Image 읽기
        image_path = os.path.join(self.root_dir, self.annotation_df.iloc[idx, 1]) #use image path column (index = 1) in csv file
        image = cv2.imread(image_path) # read image by cv2
        #### 이미지를 Channel순서를 변경
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert from BGR to RGB for matplotlib
        #### 이미지 Transform
        if self.transform:
            image = self.transform(image)

        #### 이미지에 대한 추가 정보 확인
        class_name = self.annotation_df.iloc[idx, 2] # use class name column (index = 2) in csv file
        class_index = self.annotation_df.iloc[idx, 3] # use class index column (index = 3) in csv file

        return image, class_name, class_index

In [ ]:
# https://pytorch.org/vision/stable/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py
from torchvision import transforms
import matplotlib.pyplot as plt
import random

transform = transforms.Compose([
    transforms.ToTensor(), # PIL Image를 Tensor로 변경
    transforms.Resize((100,100), antialias=True ), # 크기 변경
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # Normalize
    transforms.Grayscale(), # Gray Scale로 변경
    transforms.RandomAdjustSharpness(sharpness_factor=2), # randomly adjusts the sharpness
    transforms.RandomAutocontrast() # 임의로 Contrast변경
])

##### 실습 : train_dataset 선언
train_dataset = semiconductorDataset(csv_file='train.csv', root_dir="", transform=transform)

# Batch 단위 학습을 위한 DataLoader 선언
 - Dataset내부의 샘플들을 batch 크기로 추출
 - Batch Size는 1step에 들어간 데이터의 개수
 - Epoch 마다 데이터를 섞어(Shuffle) Overfitting을 방지
 - 병렬처리를 지원하여 데이터 검색 속도를 향상


local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).

In [ ]:
#### 실습 : torch.utils.DataLoader를 이용하여 데이터를 load. Batch 크기를 10으로 한다.
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=10, shuffle=True, num_workers=2)

for i, data in enumerate(train_dataloader):
  images, class_name, labels = data
  print(images.shape, labels.shape)

  # 5번만 데이터를 load하고 멈춘다
  if i > 3:
    break

torch.Size([10, 1, 100, 100]) torch.Size([10])
torch.Size([10, 1, 100, 100]) torch.Size([10])
torch.Size([10, 1, 100, 100]) torch.Size([10])
torch.Size([10, 1, 100, 100]) torch.Size([10])
torch.Size([10, 1, 100, 100]) torch.Size([10])


# VGG6를 이용하여 학습을 수행한다
![](https://drive.google.com/uc?export=view&id=1vLrvhxczx1ZCOH05cxzzESylaOaF1Uj2)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

class VGG_BatchNormTorch(nn.Module):
  def __init__(self, in_channels, out_features):
    super(VGG_BatchNormTorch, self).__init__()

    ###### 실습 : Gray이미지 이므로 in_channel =1 로 선언한다
    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=3, padding=1)
    self.norm1 = torch.nn.BatchNorm2d(32)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
    self.norm2 = torch.nn.BatchNorm2d(32)

    self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
    self.norm3 = torch.nn.BatchNorm2d(64)

    self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
    self.norm4 = torch.nn.BatchNorm2d(64)

    self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    #### 실습 : Batch Norm 을 Pytorch의 값으로 변경한다
    self.norm5 = torch.nn.BatchNorm2d(128)
    self.conv6 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
    #### 실습 : Batch Norm 을 Pytorch의 값으로 변경한다
    self.norm6 = torch.nn.BatchNorm2d(128)

    self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2) #Maxpooling layer to change feature size
    self.avg_pool = nn.AdaptiveAvgPool2d(output_size = (1, 1)) #Note that average pooling layer is not adopted in original VGG architecture. We use average pooling layer to make the architecture for experiment simple.

    ###### 실습 : 11개의 Label이 있으므로 out_features=11로 선언한다
    self.fc = nn.Linear(in_features=128, out_features=out_features)

  def forward(self, x):
    #### 실습 : Batch Norm을 Convolution 이후에 선언
    x = self.norm1(self.conv1(x))
    x = F.relu(x)
    x = self.norm2(self.conv2(x))
    x = F.relu(x)
    x = self.max_pool(x)

    x = self.norm3(self.conv3(x))
    x = F.relu(x)
    x = self.norm4(self.conv4(x))
    x = F.relu(x)
    x = self.max_pool(x)

    x = self.norm5(self.conv5(x))
    x = F.relu(x)
    x = self.norm6(self.conv6(x))
    x = F.relu(x)

    x = self.avg_pool(x)
    x = x.view(-1, 128)
    x = self.fc(x)
    return x

In [ ]:
def train(model, data_loader, criterion, optimizer, n_epoch):

  model.train() #
  for epoch in range(n_epoch):
    running_loss = 0
    ### 실습 : Custom dataloader 에서 값을 가져온다
    for i, (images, _, labels) in enumerate(data_loader):
      images, labels = images.cuda(), labels.cuda()
      outputs = model(images)
      loss = criterion(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      if (i + 1) % 100 == 0:
        print('iteration: [{}/{}]'.format(i + 1, len(data_loader)))

    print('Epoch {}, loss = {:.3f}'.format(epoch + 1, running_loss/len(data_loader)))

In [ ]:
def eval(model, data_loader):

  model.eval()
  total = 0
  correct = 0

  preds = []
  trues = []

  with torch.no_grad():
    ### 실습 : Custom dataloader 에서 값을 가져온다
    for images, _,  labels in data_loader:
      images, labels = images.cuda(), labels.cuda()
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

      preds.append(predicted.detach().cpu().numpy())
      trues.append(labels.detach().cpu().numpy())

    accuracy = 100 * correct / total

  print('Test Accuracy: {}%'.format(accuracy))
  return preds, trues


In [ ]:
def reset_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
reset_seed(2020)
criterion = nn.CrossEntropyLoss()
vgg_batchnorm_model = VGG_BatchNormTorch(in_channels=1, out_features=11).to("cuda")
optimizer = optim.Adam(params=vgg_batchnorm_model.parameters())

train(vgg_batchnorm_model, train_dataloader, criterion, optimizer, n_epoch=50)

Epoch 1, loss = 2.419
Epoch 2, loss = 2.259
Epoch 3, loss = 2.087
Epoch 4, loss = 2.034
Epoch 5, loss = 1.968
Epoch 6, loss = 1.843
Epoch 7, loss = 1.745
Epoch 8, loss = 1.699
Epoch 9, loss = 1.588
Epoch 10, loss = 1.443
Epoch 11, loss = 1.361
Epoch 12, loss = 1.246
Epoch 13, loss = 1.287
Epoch 14, loss = 1.217
Epoch 15, loss = 1.179
Epoch 16, loss = 1.167
Epoch 17, loss = 1.094
Epoch 18, loss = 1.014
Epoch 19, loss = 0.995
Epoch 20, loss = 0.961
Epoch 21, loss = 0.843
Epoch 22, loss = 0.794
Epoch 23, loss = 0.784
Epoch 24, loss = 0.674
Epoch 25, loss = 0.640
Epoch 26, loss = 0.603
Epoch 27, loss = 0.596
Epoch 28, loss = 0.491
Epoch 29, loss = 0.457
Epoch 30, loss = 0.420
Epoch 31, loss = 0.399
Epoch 32, loss = 0.333
Epoch 33, loss = 0.351
Epoch 34, loss = 0.339
Epoch 35, loss = 0.291
Epoch 36, loss = 0.274
Epoch 37, loss = 0.222
Epoch 38, loss = 0.240
Epoch 39, loss = 0.216
Epoch 40, loss = 0.236
Epoch 41, loss = 0.174
Epoch 42, loss = 0.216
Epoch 43, loss = 0.199
Epoch 44, loss = 0.1

# 정확도를 평가해 봅시다
* 데이터가 너무 적어서 학습 데이터에 대한 정확도를 봅니다

In [ ]:
preds, trues = eval(vgg_batchnorm_model, train_dataloader)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

Test Accuracy: 59.5959595959596%


In [ ]:
prediction_label = np.stack(preds[0:-1]).flatten()
train_labels = np.stack(trues[0:-1]).flatten()
prediction_label, train_labels

(array([ 8,  3,  3,  2,  0,  8,  7,  8,  4,  5,  8,  5, 10,  5,  5,  0,  3,
         0,  9,  3,  8,  4,  5,  7,  5,  2,  7,  2, 10,  2,  9,  2,  2,  8,
         7, 10,  2,  5,  5,  5,  8,  8,  2,  7,  7,  2,  8, 10,  8,  8,  5,
         4,  5, 10,  0,  7,  5,  0, 10,  7,  5,  7,  5,  3,  5,  4,  9,  7,
         9,  7,  2,  5, 10,  5,  2,  2,  5,  7,  4,  5,  2,  7,  5,  5,  3,
         9,  2,  7,  8,  9,  8,  4,  9,  8,  3,  1,  5,  9,  5,  1, 10,  5,
         8,  2,  0,  9,  7,  9,  8,  8,  0, 10,  5,  2,  4,  8,  7,  8,  0,
         4,  7,  5,  5,  7,  3,  2, 10,  3,  1,  3,  5, 10,  4,  9,  4,  3,
         2,  3, 10,  2, 10,  3,  3,  5,  2,  0, 10,  9,  7,  7,  8,  9,  5,
         9,  4,  2, 10,  7,  2,  5,  7,  7,  9,  5,  8,  0,  4,  9,  2,  8,
         4,  5,  7,  6,  2,  4,  7,  9,  8,  4,  3,  9,  2,  7,  2,  5,  9,
         7,  7, 10]),
 array([ 1,  6,  6,  1,  0,  8,  7,  8,  4,  5,  8,  5, 10,  5,  5,  2,  3,
         0,  9,  3,  7,  4,  0,  7,  5,  2,  7,  2, 10,  1,  0,  2

In [ ]:
########### 실습 : 분류 성능을 평가해 봅니다.
from sklearn.metrics import classification_report
print(classification_report(train_labels, prediction_label))

              precision    recall  f1-score   support

           0       0.50      0.28      0.36        18
           1       1.00      0.18      0.30        17
           2       0.54      0.74      0.62        19
           3       0.73      0.69      0.71        16
           4       0.93      0.82      0.87        17
           5       0.44      0.88      0.59        17
           6       1.00      0.06      0.11        17
           7       0.50      0.82      0.62        17
           8       0.35      0.44      0.39        18
           9       0.58      0.61      0.59        18
          10       1.00      1.00      1.00        16

    accuracy                           0.59       190
   macro avg       0.69      0.59      0.56       190
weighted avg       0.68      0.59      0.56       190



# RGB로 학습 후 평가해 본다

In [ ]:
# https://pytorch.org/vision/stable/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py
from torchvision import transforms
import matplotlib.pyplot as plt
import random

transform = transforms.Compose([
    transforms.ToTensor(), # PIL Image를 Tensor로 변경
    transforms.Resize((100,100), antialias=True), # 크기 변경
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # Normalize
    # transforms.Grayscale(), # Gray Scale로 변경
    transforms.RandomAdjustSharpness(sharpness_factor=2), # randomly adjusts the sharpness
    transforms.RandomAutocontrast() # 임의로 Contrast변경
])

train_dataset = semiconductorDataset(csv_file='train.csv', root_dir="", transform=transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=10, shuffle=True, num_workers=2)

In [ ]:
reset_seed(2020)
criterion = nn.CrossEntropyLoss()
############### 실습 : 입력 Channel의 개수는 3
vgg_batchnorm_model = VGG_BatchNormTorch(in_channels=3, out_features=11).to("cuda")
optimizer = optim.Adam(params=vgg_batchnorm_model.parameters())

train(vgg_batchnorm_model, train_dataloader, criterion, optimizer, n_epoch=40)

Epoch 1, loss = 2.355
Epoch 2, loss = 2.175
Epoch 3, loss = 2.043
Epoch 4, loss = 2.013
Epoch 5, loss = 1.979
Epoch 6, loss = 1.940
Epoch 7, loss = 1.963
Epoch 8, loss = 1.943
Epoch 9, loss = 1.831
Epoch 10, loss = 1.744
Epoch 11, loss = 1.613
Epoch 12, loss = 1.581
Epoch 13, loss = 1.603
Epoch 14, loss = 1.505
Epoch 15, loss = 1.454
Epoch 16, loss = 1.518
Epoch 17, loss = 1.431
Epoch 18, loss = 1.359
Epoch 19, loss = 1.335
Epoch 20, loss = 1.270
Epoch 21, loss = 1.223
Epoch 22, loss = 1.171
Epoch 23, loss = 1.240
Epoch 24, loss = 1.165
Epoch 25, loss = 1.178
Epoch 26, loss = 1.125
Epoch 27, loss = 1.068
Epoch 28, loss = 1.081
Epoch 29, loss = 1.050
Epoch 30, loss = 1.068
Epoch 31, loss = 0.994
Epoch 32, loss = 0.901
Epoch 33, loss = 0.861
Epoch 34, loss = 0.960
Epoch 35, loss = 0.912
Epoch 36, loss = 0.848
Epoch 37, loss = 0.928
Epoch 38, loss = 0.858
Epoch 39, loss = 0.888
Epoch 40, loss = 0.805


In [ ]:
preds, trues = eval(vgg_batchnorm_model, train_dataloader)

Test Accuracy: 69.6969696969697%


In [ ]:
prediction_label = np.stack(preds[0:-1]).flatten()
train_labels = np.stack(trues[0:-1]).flatten()
prediction_label, train_labels

(array([ 4,  9,  3,  7,  2,  9,  5, 10,  4,  2,  6,  2,  2,  4,  6,  9,  6,
         6,  1,  9,  6,  1,  2,  0, 10,  5,  9,  9,  4,  1,  2,  4,  6,  2,
         6,  2, 10,  3,  2,  2,  7,  3,  9,  5,  9,  6,  5,  9,  6,  3,  2,
         9,  6,  6,  4, 10,  3,  9,  9,  3,  3,  2,  6,  9,  2,  5,  9,  2,
         4,  5,  3, 10,  1,  6,  2,  0,  0,  0,  2, 10,  2,  2,  4,  5,  3,
         2,  2,  3,  3,  0,  5,  2,  6, 10, 10,  2,  6,  9,  3,  3,  0,  2,
         3,  9,  9,  2,  9,  9,  3,  2,  0,  4,  9,  2,  1,  1,  6,  4,  9,
         7,  0,  9,  2,  2,  2,  9, 10,  5,  9,  0, 10,  5,  9,  2,  2,  9,
         1,  9,  3,  3, 10,  0, 10,  7,  2,  2, 10,  9,  2,  9,  5,  2,  2,
         2,  9,  2,  4,  9,  0,  2,  4,  2,  5,  0,  6,  9,  5,  4, 10,  9,
         9, 10,  9,  0,  9,  9,  4,  5,  4, 10,  4,  2,  0,  2,  9,  7,  1,
         5,  2, 10]),
 array([ 4,  9,  3,  8,  2,  8,  5, 10,  4,  1,  6,  7,  1,  4,  6,  7,  6,
         6,  1,  0,  6,  1,  2,  2, 10,  5,  7,  7,  4,  1,  1,  4